In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import string

In [2]:
df = pd.read_csv('Tweets.csv')
df.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [3]:
df.columns

Index(['tweet_id', 'airline_sentiment', 'airline_sentiment_confidence',
       'negativereason', 'negativereason_confidence', 'airline',
       'airline_sentiment_gold', 'name', 'negativereason_gold',
       'retweet_count', 'text', 'tweet_coord', 'tweet_created',
       'tweet_location', 'user_timezone'],
      dtype='object')

In [4]:
df.describe()

,tweet_id,airline_sentiment_confidence,negativereason_confidence,retweet_count
count,1.464000e+04,14640.000000,10522.000000,14640.000000
mean,5.692184e+17,0.900169,0.638298,0.082650
std,7.791112e+14,0.162830,0.330440,0.745778
min,5.675883e+17,0.335000,0.000000,0.000000
25%,5.685592e+17,0.692300,0.360600,0.000000
50%,5.694779e+17,1.000000,0.670600,0.000000
75%,5.698905e+17,1.000000,1.000000,0.000000
max,5.703106e+17,1.000000,1.000000,44.000000


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14640 entries, 0 to 14639
Data columns (total 15 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   tweet_id                      14640 non-null  int64  
 1   airline_sentiment             14640 non-null  object 
 2   airline_sentiment_confidence  14640 non-null  float64
 3   negativereason                9178 non-null   object 
 4   negativereason_confidence     10522 non-null  float64
 5   airline                       14640 non-null  object 
 6   airline_sentiment_gold        40 non-null     object 
 7   name                          14640 non-null  object 
 8   negativereason_gold           32 non-null     object 
 9   retweet_count                 14640 non-null  int64  
 10  text                          14640 non-null  object 
 11  tweet_coord                   1019 non-null   object 
 12  tweet_created                 14640 non-null  object 
 13  t

In [6]:
df['airline'].value_counts()

airline
United            3822
US Airways        2913
American          2759
Southwest         2420
Delta             2222
Virgin America     504
Name: count, dtype: int64

In [7]:
df = df.drop(columns =['tweet_id','airline_sentiment_confidence','negativereason', 'negativereason_confidence', 'airline','airline_sentiment_gold', 'name', 'negativereason_gold','retweet_count','tweet_coord', 'tweet_created','tweet_location', 'user_timezone'])

In [8]:
df.head()

,airline_sentiment,text
0,neutral,@VirginAmerica What @dhepburn said.
1,positive,@VirginAmerica plus you've added commercials t...
2,neutral,@VirginAmerica I didn't today... Must mean I n...
3,negative,@VirginAmerica it's really aggressive to blast...
4,negative,@VirginAmerica and it's a really big bad thing...


In [9]:
data = df.rename(columns={'airline_sentiment':"Class"})

In [10]:
data.head()

,Class,text
0,neutral,@VirginAmerica What @dhepburn said.
1,positive,@VirginAmerica plus you've added commercials t...
2,neutral,@VirginAmerica I didn't today... Must mean I n...
3,negative,@VirginAmerica it's really aggressive to blast...
4,negative,@VirginAmerica and it's a really big bad thing...


In [11]:
data['text'][0]

'@VirginAmerica What @dhepburn said.'

In [12]:
data['text'][100]

'@VirginAmerica trying to add my boy Prince to my ressie. SF this Thursday @VirginAmerica from LAX http://t.co/GsB2J3c4gM'

## Text Preprocessing

In [13]:
from nltk.stem import PorterStemmer 

In [14]:
ps = PorterStemmer()

In [15]:
airLines = ['@United','@US Airways','@American','@Southwest','@Delta','@VirginAmerica']
def processs(text):
    text = [ ps.stem(words) for words in text.split() if words not in airLines]
    text = " ".join(text).lower()
    text = re.sub('\[.*?\]' ,'' ,text)
    text = re.sub('\\W' ," ",text)
    text = re.sub('https?://\S+|www\.\S+' ,'',text)
    text = re.sub('<.*?>+' ,'',text)
    text = re.sub('[%s]' %re.escape(string.punctuation),'',text)
    text = re.sub('\n' ,'',text)
    text = re.sub('\w*\d\w*','',text)
    
    return text

In [16]:
print(processs(data['text'][100]))

tri to add my boy princ to my ressie  sf thi thursday from lax http   t co 


In [17]:
data['text'] = data['text'].apply(processs)

## Convert to the vector

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf = TfidfVectorizer(stop_words="english")

In [19]:
from sklearn.linear_model import LogisticRegression

lr =LogisticRegression()

In [20]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

In [21]:
data['Class'] = le.fit_transform(data['Class'])

data.head()

,Class,text
0,1,what dhepburn said
1,2,plu you v ad commerci to the experience tacky
2,1,i didn t today must mean i need to take ano...
3,0,it realli aggress to blast obnoxi entertainm...
4,0,and it a realli big bad thing about it


In [22]:
from sklearn.model_selection import train_test_split

x = data['text']
y = data['Class']

x_train , x_test ,y_train , y_test = train_test_split(x , y , test_size=0.23 ,random_state=3)

In [23]:
x_train_vector = tf.fit_transform(x_train)
x_test_vector = tf.transform(x_test)

In [24]:
print(x_train_vector)

  (0, 3123)	0.5793078047390885
  (0, 1218)	0.4553289080487616
  (0, 9077)	0.581523899979058
  (0, 7720)	0.3448304026860845
  (1, 4649)	0.3967274405844302
  (1, 6291)	0.21328423335483868
  (1, 4018)	0.3767928784898819
  (1, 1532)	0.3701392131295309
  (1, 9046)	0.2502907690843046
  (1, 2253)	0.2880615621248578
  (1, 3626)	0.3204053030549901
  (1, 7400)	0.5016021714461755
  (1, 305)	0.13510008891493122
  (2, 7141)	0.4609204286758599
  (2, 4797)	0.33721219695774296
  (2, 7748)	0.5543890961955322
  (2, 359)	0.336993675038027
  (2, 7719)	0.45758378040949527
  (2, 7720)	0.20867527300141325
  (3, 3971)	0.14033361213838008
  (3, 2938)	0.3184719514659361
  (3, 3424)	0.3230841320226776
  (3, 4647)	0.22227041529977587
  (3, 2764)	0.34148866361183805
  (3, 7803)	0.3614917141079403
  :	:
  (11269, 835)	0.6689284019667548
  (11269, 3222)	0.6680743794479701
  (11269, 8835)	0.32590093060225095
  (11270, 9149)	0.4887855643477384
  (11270, 8983)	0.4887855643477384
  (11270, 2351)	0.4536294283648216
  (11

In [25]:
print(x_train[4983])

 southwestair find a way to cancel flight flight        


In [26]:
print(y_train[4983])

0


In [27]:
print(x_test[528])

 unit yes  a paper voucher that i got on januari  


In [28]:
print(y_test[528])

1


In [29]:
lr.fit(x_train_vector , y_train)

C:\Users\darsh\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [30]:
print("Accuracy :- ",lr.score(x_train_vector , y_train))

Accuracy :-  0.8685237757274663


In [31]:
print("Accuracy :- ",lr.score(x_test_vector , y_test))

Accuracy :-  0.7776128266033254


## Decision Tree 

In [32]:
from sklearn.tree import DecisionTreeClassifier

dc = DecisionTreeClassifier()

In [33]:
dc.fit(x_train_vector , y_train)

DecisionTreeClassifier()

In [34]:
print("Accuracy :- ",dc.score(x_test_vector , y_test))

Accuracy :-  0.6707244655581948


In [35]:
dc.predict(x_test_vector[528])

array([1])

## Random Forest

In [36]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()

In [37]:
rf.fit(x_train_vector , y_train)

RandomForestClassifier()

In [38]:
print("Accuracy :- ",rf.score(x_test_vector , y_test))

Accuracy :-  0.747624703087886
